In [9]:
using QuantumOptics
using PyPlot
PyPlot.pygui(true)

true

In [10]:
# Parameters
γ2 = 0.2 # in MHz, intermediate state decay to |1>
γ3 = 0.01 # in MHz, Rydberg decay to |4> (some other states we don't care about). 0.01MHz ~ 100us
Ω1 = Ω2 = 5     # in MHz 
Δ = 0     # in MHz
tmax = 8  # in μs
dt = 0.001
tlist = [0:dt:tmax;]

8001-element Vector{Float64}:
 0.0
 0.001
 0.002
 0.003
 0.004
 0.005
 0.006
 0.007
 0.008
 0.009
 ⋮
 7.992
 7.993
 7.994
 7.995
 7.996
 7.997
 7.998
 7.999
 8.0

In [11]:
# Basis and operators
b = NLevelBasis(4)
σ1 = transition(b, 1, 2)  # |1><2|
σ2 = transition(b, 2, 3)  # |2><3|
Ryd = transition(b, 4, 3)  # |4><3|, decay out of the system
proj1 = transition(b, 1, 1) # |1><1|
proj2 = transition(b, 2, 2) # |2><2|
proj3 = transition(b, 3, 3)     # |3><3|

Operator(dim=4x4)
  basis: NLevel(N=4)sparse([3], [3], ComplexF64[1.0 + 0.0im], 4, 4)

In [12]:
# Hamiltonian and jump operators
J = [γ2*σ1+γ3*Ryd];
Jdagger = [γ2*dagger(σ1)+γ3*dagger(Ryd)]; 

# Initial state
ψ0 = nlevelstate(b, 1)

Ket(dim=4)
  basis: NLevel(N=4)
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

In [13]:
# Define the Gaussian function
function gaussian(t, mu, sigma, A)
    return A * exp(-(t - mu)^2 / (2 * sigma^2))
end

mu1 = 4.1  # center of the peak in us
mu2 = 3  # center of the peak in us
sigma = 0.8 # span of in us
tau = 0.01  # time resolution in us

white_noise = 0.1

# Define the driving term functions
function H(t, psi)
    # Round the current time to the nearest multiple of tau
    t_rounded = round(t / tau) * tau
    return 0.5*(σ1+dagger(σ1))*gaussian(t_rounded, mu1, sigma, Ω1)+0.5*(σ2+dagger(σ2))*gaussian(t_rounded, mu2, sigma, Ω2)+Δ*proj2+1im*γ3*proj3, J, Jdagger
end

# h_mu2 = H(mu2,ψ0)

# Plots
figure(figsize=(6, 3))
plot(tlist, gaussian.(tlist, mu1, sigma, Ω1), label="1-2")
plot(tlist, gaussian.(tlist, mu2, sigma, Ω2), label="2-3")
axis([0, tmax, 0, Ω1])
legend()
PyPlot.show(block=false)

#=
function H_pump2(t, psi)
    mu = 3e-6  # center of the peak at 3us
    sigma = 1e-6  # span of 1us
    tau = 1e-9  # time resolution of 1ns
    # Round the current time to the nearest multiple of tau
    t_rounded = round(t / tau) * tau
    return dagger(σ2)*gaussian(t_rounded, mu, sigma, Ω2)
end
=#

In [14]:
# Expectation values
function calc_pops(t, ρ)
    p1 = real(expect(proj1, ρ))
    p2 = real(expect(proj2, ρ))
    p3 = real(expect(proj3, ρ))
    return p1, p2, p3
end

calc_pops (generic function with 1 method)

In [15]:
# Time evolution
tout, pops = timeevolution.master_dynamic(tlist, ψ0, H; fout=calc_pops)

([0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009  …  7.991, 7.992, 7.993, 7.994, 7.995, 7.996, 7.997, 7.998, 7.999, 8.0], Tuple{Any, Any, Any}[(1.0, 0.0, 0.0), (1.0, 4.217536228708562e-17, 4.80012454600727e-24), (0.9999999999999998, 1.6790724003344744e-16, 1.879752878264403e-23), (0.9999999999999997, 3.760461130666339e-16, 4.1398632401660976e-23), (0.9999999999999993, 6.65498185880681e-16, 7.202435348657848e-23), (0.999999999999999, 1.0352253876361652e-15, 1.1011010851268826e-22), (0.9999999999999986, 1.4842453721601542e-15, 1.5510581234705212e-22), (0.999999999999998, 2.0116315179462078e-15, 2.0647587824850123e-22), (0.9999999999999973, 2.6165129281544424e-15, 2.636992178676399e-22), (0.9999999999999967, 3.2980744306114464e-15, 3.2626924124683974e-22)  …  (0.004105902145354073, 0.00438852858871656, 0.9910755091271779), (0.004106077608804247, 0.0043883531262680076, 0.9910754100186303), (0.004106253065915406, 0.004388177670146713, 0.9910753109101044), (0.004106428516

In [16]:
# Reshape pops
p1 = [p[1] for p=pops]
p2 = [p[2] for p=pops]
p3 = [p[3] for p=pops]
final_p3 = round(p3[end], digits=4)
final_p3_str = string(final_p3)
# Plots
figure(figsize=(6, 3))
plot(tout, p1, label="Initial ground state")
plot(tout, p2, "--", label="Intermediate state")
plot(tout, p3, label="Excited state")
title("Final p3: $final_p3_str")
axis([0, tmax, 0, 1])
legend()
PyPlot.show(block=false)